In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import datasets
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from matplotlib import pyplot as plt

In [9]:
# let's define some useful functions here
def z_score(vec):
    
    return (vec - np.mean(vec))/np.std(vec)

def pearson_correlation(vec1, vec2):
    
    z1 = z_score(vec1)
    z2 = z_score(vec2)
    
    assert z1.shape[0] == z2.shape[0], print("Corr function requires two vectors of the same shape.")
    
    n = z1.shape[0]
    
    return np.dot(z1, z2)/n

In [16]:
# load dataset
iris = datasets.load_iris()
X = iris.data[:, :1]
y = iris.data[:, 3]
print(f"Number of predictors {X.shape[1]}")
print(f"Number of trials {X.shape[0]}")

Number of predictors 1
Number of trials 150


In [43]:
bias = np.ones((X.shape[0],1))
X_with_bias = np.hstack((X,bias))
n = X.shape[0]
num_predictors = X.shape[1]
residuals_df = (n-1) - num_predictors

In [19]:
X_pd = pd.DataFrame(np.hstack((X_with_bias,np.expand_dims(y,axis=-1))), columns=['pred1', 'bias', 'y'])

In [21]:
model = smf.ols("y~pred1", data=X_pd)
model_fit = model.fit()
print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     299.2
Date:                Sun, 07 May 2023   Prob (F-statistic):           2.33e-37
Time:                        02:59:00   Log-Likelihood:                -88.686
No. Observations:                 150   AIC:                             181.4
Df Residuals:                     148   BIC:                             187.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.2002      0.257    -12.458      0.0

In [22]:
# y = XB
beta = np.linalg.pinv(X_with_bias)@y
y_hat = X_with_bias@beta

In [23]:
reg = LinearRegression().fit(X,y)
print(reg.coef_, reg.intercept_)

[0.75291757] -3.2002150046491913


In [39]:
# compute SS of y 
SS_total = np.sum((y - np.mean(y))**2)
SS_model = np.sum((y_hat - np.mean(y))**2)
r2 = SS_model / SS_total
print(f"R2 {np.round(r2,2)}, R2 scikit {np.round(reg.score(X,y),2)}")

R2 0.67, R2 scikit 0.67


In [57]:
print(SS_model)

57.917682179192326


In [50]:
# residual standard error
residuals = y - y_hat
res_var = np.sum(residuals**2)/residuals_df
res_se = res_var**.5
print(f"Residual standard error: {np.round(res_se,3)}")

Residual standard error: 0.44


In [56]:
# predictor standard error 
r = pearson_correlation(y, X.squeeze())
print(r)

0.8179411262715757


In [24]:
MS_model = SS_model/num_predictors
SS_residual = SS_total - SS_model # unexplained variance
MS_residual = SS_residual / ((n-1)-num_predictors)
F_stat = MS_model/MS_residual
print(F_stat)

212.4121206506138
